In [ ]:
import pandas as pd

comments_1 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments1.csv")
comments_2 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments2.csv")
comments_3 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments3.csv")
comments_4 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments4.csv")
comments_5 = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\comments5.csv")

In [41]:
df_video = pd.read_csv(r"C:\Users\Nixon Ngai Weng Yue\Downloads\dataset\videos.csv")

In [ ]:
for i, df in enumerate([comments_1, comments_2, comments_3, comments_4,comments_5], start=1):
    df["dataset_id"] = i

df_all_comments = pd.concat([comments_1, comments_2, comments_3, comments_4,comments_5], ignore_index=True)

In [ ]:
def compute_soe(video_df, comment_df, w_video_like=1, w_fav=2, w_comment=3, w_comment_like=1):
    k = int(video_df["viewCount"].median())

    video_df["publishedAt"] = pd.to_datetime(video_df["publishedAt"], errors="coerce", utc=True)

    comment_stats = comment_df.groupby("videoId").agg(
        comment_likes_sum=("likeCount", "sum"),
        comment_count=("commentId", "count")
    ).reset_index()

    merged = video_df.merge(comment_stats, on="videoId", how="left")

    merged["comment_likes_sum"] = merged["comment_likes_sum"].fillna(0)
    merged["comment_count"] = merged["comment_count"].fillna(0)

    merged["engagement"] = (
        w_video_like * merged["likeCount"] +
        w_fav * merged["favouriteCount"] +
        w_comment * merged["commentCount"] +
        w_comment_like * merged["comment_likes_sum"]
    )

    merged["denominator"] = merged["viewCount"] + k

    merged["SoE"] = merged["engagement"] / merged["denominator"]

    comment_with_soe = comment_df.merge(
        merged[["videoId", "SoE"]],
        on="videoId",
        how="left"
    )

    return comment_with_soe


In [ ]:
result_soe = compute_soe(df_video, df_all_comments)
print(result_soe.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [ ]:
def get_comment_depth(comment_id, parent_map):
    depth = 0
    parent = parent_map.get(comment_id, None)
    while parent is not None:
        depth += 1
        parent = parent_map.get(parent, None)
    return depth

def compute_depths(df):
    df = df.copy()
    if "commentId" not in df.columns or "parentCommentId" not in df.columns:
        raise KeyError("DataFrame must contain 'commentId' and 'parentCommentId' columns")
    parent_map = df.set_index("commentId")["parentCommentId"].to_dict()
    df["depth"] = df["commentId"].apply(lambda cid: get_comment_depth(cid, parent_map))
    return df

In [46]:
result_compute_depths = compute_depths(df_all_comments)
print(result_compute_depths.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [ ]:
import numpy as np
def compute_reply_factor(df, depth_weight=0.5):
    df = df.copy()
    if "depth" not in df.columns:
        df = compute_depths(df)

    reply_counts = df["parentCommentId"].value_counts().to_dict()
    df["numReplies"] = df["commentId"].map(reply_counts).fillna(0).astype(int)

    df["ReplyFactor"] = (1 + depth_weight * df["depth"]) * (1 + np.log1p(df["numReplies"]))
    return df

In [ ]:
result_reply_factor = compute_reply_factor(df_all_comments,depth_weight=0.5)
print(result_reply_factor.head())

              kind  commentId  channelId  videoId  authorId  \
0  youtube#comment    1781382      14492    74288   2032536   
1  youtube#comment     289571      14727    79618   3043229   
2  youtube#comment     569077       3314    51826    917006   
3  youtube#comment    2957962       5008    58298   1853470   
4  youtube#comment     673093      21411     1265   2584166   

                                          textOriginal  parentCommentId  \
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would rock it              NaN   
1     Apply mashed potato juice and mixed it with curd        3198066.0   
2                           69 missed calls from mars👽              NaN   
3                                                 Baaa              NaN   
4      you look like raven from phenomena raven no cap              NaN   

   likeCount                publishedAt                  updatedAt  \
0          0  2023-08-15 21:48:52+00:00  2023-08-15 21:48:52+00:00   
1          0  2023-10-02 13:08

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_relevance_by_video(video_df, comment_df, text_col="textOriginal",
                                 video_meta_cols=("title","description","tags")):
    comment_df = comment_df.copy()

    for c in ("videoId", text_col, "commentId"):
        if c not in comment_df.columns:
            raise KeyError(f"comment_df must contain column '{c}'")
    for c in ("videoId",) + video_meta_cols:
        if c not in video_df.columns:
            raise KeyError(f"video_df must contain column '{c}'")

    video_df = video_df.copy()
    video_df["video_meta"] = (
        video_df[video_meta_cols[0]].fillna("").astype(str) + " " +
        video_df[video_meta_cols[1]].fillna("").astype(str) + " " +
        video_df[video_meta_cols[2]].fillna("").astype(str)
    )
    video_meta_map = dict(zip(video_df["videoId"], video_df["video_meta"]))

    relevances = np.zeros(len(comment_df), dtype=float)

    for vid, group in comment_df.groupby("videoId", sort=False):
        idx = group.index
        video_text = str(video_meta_map.get(vid, "")).strip()
        comments = group[text_col].fillna("").astype(str).str.strip().tolist()

        if (not video_text) or all(not c for c in comments):
            continue 

        try:
            texts = comments + [video_text]   
            vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
            vectors = vectorizer.fit_transform(texts)
            comment_vecs = vectors[:-1]   
            video_vec = vectors[-1]

            sims = cosine_similarity(comment_vecs, video_vec).reshape(-1)
            sims = np.clip(sims, 0.0, 1.0)
            relevances[idx] = sims

        except ValueError:
            relevances[idx] = 0.0

    comment_df["relevance"] = relevances
    return comment_df

In [ ]:
result_relevance = calculate_relevance_by_video(df_video,df_all_comments)
print(result_relevance.head())

              kind  commentId  channelId  videoId  authorId  \
0  youtube#comment    1781382      14492    74288   2032536   
1  youtube#comment     289571      14727    79618   3043229   
2  youtube#comment     569077       3314    51826    917006   
3  youtube#comment    2957962       5008    58298   1853470   
4  youtube#comment     673093      21411     1265   2584166   

                                          textOriginal  parentCommentId  \
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would rock it              NaN   
1     Apply mashed potato juice and mixed it with curd        3198066.0   
2                           69 missed calls from mars👽              NaN   
3                                                 Baaa              NaN   
4      you look like raven from phenomena raven no cap              NaN   

   likeCount                publishedAt                  updatedAt  \
0          0  2023-08-15 21:48:52+00:00  2023-08-15 21:48:52+00:00   
1          0  2023-10-02 13:08

In [ ]:
def compute_time_norm(video_df, comment_df, tz="UTC"):
    today = pd.Timestamp.now(tz=tz)

    video_df = video_df.copy()
    comment_df = comment_df.copy()
    video_df["publishedAt"] = pd.to_datetime(video_df["publishedAt"], errors="coerce", utc=True)
    comment_df["publishedAt"] = pd.to_datetime(comment_df["publishedAt"], errors="coerce", utc=True)

    comment_df["days_since_comment"] = (today - comment_df["publishedAt"]).dt.days

    max_days_comment = comment_df["days_since_comment"].max()

    if max_days_comment == 0 or pd.isna(max_days_comment):
        comment_df["timeNorm"] = 1.0
    else:
        comment_df["timeNorm"] = 1 - (comment_df["days_since_comment"] / max_days_comment)
        comment_df["timeNorm"] = comment_df["timeNorm"].fillna(1.0)

    return comment_df


In [56]:
result_time_norm = compute_time_norm(df_video,df_all_comments,tz="UTC")
print(result_time_norm.head(20))

               kind  commentId  channelId  videoId  authorId  \
0   youtube#comment    1781382      14492    74288   2032536   
1   youtube#comment     289571      14727    79618   3043229   
2   youtube#comment     569077       3314    51826    917006   
3   youtube#comment    2957962       5008    58298   1853470   
4   youtube#comment     673093      21411     1265   2584166   
5   youtube#comment    1525723      18073    69091   1915402   
6   youtube#comment    3079259       2988    91785    885833   
7   youtube#comment     579871      45729    66477   2775194   
8   youtube#comment    4596873      18670    88816   1040123   
9   youtube#comment     910365      25235    31185   1092265   
10  youtube#comment     779733      36205    72492   3158722   
11  youtube#comment    1204214      33644    86518   1756840   
12  youtube#comment     957058      20162    75828   1548100   
13  youtube#comment    4672340      29550    53751   1064748   
14  youtube#comment    2797490       477

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold

def train_weights(df, target_col="likeCount", log_transform=True, cv_folds=5):
    required_cols = ["SoE", "ReplyFactor", "relevance"]

    for col in required_cols + [target_col]:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' is missing from dataframe")

    X = df[required_cols].fillna(0)
    y = df[target_col].fillna(0)

    if log_transform:
        y = np.log1p(y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    model = LinearRegression()
    model.fit(X_scaled, y)

    cv = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_scaled, y, cv=cv, scoring="r2")

    results = pd.DataFrame([
        {"Item": "Model", "Value": model},
        {"Item": "Scaler", "Value": scaler},
        {"Item": "Intercept", "Value": model.intercept_},
        {"Item": "CV_Scores", "Value": cv_scores},
        {"Item": "CV_Mean", "Value": np.mean(cv_scores)},
    ])

    for feature, weight in zip(required_cols, model.coef_):
        results = pd.concat([results, pd.DataFrame([{"Item": feature, "Value": weight}])],
                            ignore_index=True)

    return results


In [ ]:
def merge_all_results(soe_results_df, reply_factor_results_df, relevance_relevance_df,time_norm_df):
    merged = soe_results_df.copy()
    merged = merged.merge(
        reply_factor_results_df[["commentId", "ReplyFactor"]],
        on="commentId",
        how="left"
    )

    merged = merged.merge(
        relevance_relevance_df[["commentId", "relevance"]],
        on="commentId",
        how="left"
    )

    merged = merged.merge(
        time_norm_df[["commentId", "timeNorm"]],
        on="commentId",
        how="left"
    )

    return merged


In [ ]:
merged_df = merge_all_results(result_soe, result_reply_factor, result_relevance,result_time_norm)
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4725012 entries, 0 to 4725011
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   kind             object 
 1   commentId        int64  
 2   channelId        int64  
 3   videoId          int64  
 4   authorId         int64  
 5   textOriginal     object 
 6   parentCommentId  float64
 7   likeCount        int64  
 8   publishedAt      object 
 9   updatedAt        object 
 10  dataset_id       int64  
 11  SoE              float64
 12  ReplyFactor      float64
 13  relevance        float64
 14  timeNorm         float64
dtypes: float64(5), int64(6), object(4)
memory usage: 540.7+ MB
None


In [ ]:
results_weight = train_weights(merged_df, target_col="likeCount")
soe_value = results_weight.loc[results_weight["Item"] == "SoE", "Value"].values[0]
print(soe_value)
print(results_weight.columns)


0.008940814968675291
Index(['Item', 'Value'], dtype='object')


In [ ]:
def compute_final_score(merged_df, weights):
    """
    Compute final score using learned weights from regression.

    Formula:
        FinalScore = intercept + (a*SoE + b*ReplyFactor + c*Relevance) * (TimeNorm if enabled)

    Parameters:
        merged_df : pd.DataFrame
            Must contain columns: SoE, ReplyFactor, relevance
        weights : dict
            Learned weights, e.g. {"SoE": 0.0089, "ReplyFactor": 0.3491, "relevance": 0.0269}
        intercept : float
            Learned intercept (default=0.0 if ignored)
        use_time_norm : bool
            Whether to multiply by TimeNorm if available in merged_df
    """
    soe_weight_value = weights.loc[weights["Item"] == "SoE", "Value"].values[0]
    reply_factor_weight_value = weights.loc[weights["Item"] == "ReplyFactor", "Value"].values[0]
    relevance_weight_value = weights.loc[weights["Item"] == "relevance", "Value"].values[0]

    score = (
        soe_weight_value * merged_df["SoE"]
        + reply_factor_weight_value * merged_df["ReplyFactor"]
        +relevance_weight_value * merged_df["relevance"]
    )

    score = score * merged_df["timeNorm"]

    merged_df["FinalScore"] = score
    return merged_df

In [ ]:
results_final_score = compute_final_score(merged_df, results_weight)
print(results_final_score.describe())

          commentId     channelId       videoId      authorId  \
count  4.725012e+06  4.725012e+06  4.725012e+06  4.725012e+06   
mean   2.362507e+06  2.677102e+04  4.699673e+04  1.820729e+06   
std    1.363995e+06  1.503666e+04  2.593627e+04  1.053868e+06   
min    0.000000e+00  1.000000e+00  0.000000e+00  0.000000e+00   
25%    1.181254e+06  1.449200e+04  2.597800e+04  9.060968e+05   
50%    2.362506e+06  2.542500e+04  4.709600e+04  1.813632e+06   
75%    3.543761e+06  4.061800e+04  6.944500e+04  2.731818e+06   
max    4.725015e+06  5.367700e+04  9.285400e+04  3.659440e+06   

       parentCommentId     likeCount    dataset_id           SoE  \
count     5.161350e+05  4.725012e+06  4.725012e+06  4.471305e+06   
mean      2.623663e+06  1.012744e+01  2.883605e+00  5.110901e-02   
std       1.215428e+06  5.444689e+02  1.367501e+00  6.220686e-02   
min       5.161510e+05  0.000000e+00  1.000000e+00  0.000000e+00   
25%       1.569545e+06  0.000000e+00  2.000000e+00  3.014186e-02   
50%   

In [ ]:
def compute_z_score(final_score_df):
    mean_final_score = final_score_df["FinalScore"].mean()
    std_dev_final_score = final_score_df["FinalScore"].std()

    if std_dev_final_score == 0:
        final_score_df["z_score"] = 0
    else:
        final_score_df["z_score"] = (final_score_df["FinalScore"] - mean_final_score) / std_dev_final_score

    final_score_df["QualityCategory"] = pd.cut(
        final_score_df["z_score"],
        bins=[-float("inf"), -1, 1, float("inf")],
        labels=["Low Quality", "Average Quality", "High Quality"]
    )

    return final_score_df

In [ ]:
results_z_score = compute_z_score(results_final_score)
print(results_z_score)

                    kind  commentId  channelId  videoId  authorId  \
0        youtube#comment    1781382      14492    74288   2032536   
1        youtube#comment     289571      14727    79618   3043229   
2        youtube#comment     569077       3314    51826    917006   
3        youtube#comment    2957962       5008    58298   1853470   
4        youtube#comment     673093      21411     1265   2584166   
...                  ...        ...        ...      ...       ...   
4725007  youtube#comment     561138      38699    90586    908808   
4725008  youtube#comment     252294      28584    61196   3255355   
4725009  youtube#comment    2861216      51200     7448    878070   
4725010  youtube#comment    4145588      38313    88044   2609812   
4725011  youtube#comment     793854      16475    52479   1713655   

                                                                                                                                                                           

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
# Download once
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to C:\Users\Nixon Ngai
[nltk_data]     Weng Yue\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
df_sentiment = df_all_comments.copy()

sia = SentimentIntensityAnalyzer()

df_sentiment["sentiment_score"] = df_sentiment["textOriginal"].fillna("").astype(str).apply(
    lambda x: sia.polarity_scores(x)["compound"]
)

df_sentiment["sentiment"] = df_sentiment["sentiment_score"].apply(
    lambda x: "POSITIVE" if x > 0.05 else ("NEGATIVE" if x < -0.05 else "NEUTRAL")
)

print(df_sentiment.head())

              kind  commentId  channelId  videoId  authorId  \
0  youtube#comment    1781382      14492    74288   2032536   
1  youtube#comment     289571      14727    79618   3043229   
2  youtube#comment     569077       3314    51826    917006   
3  youtube#comment    2957962       5008    58298   1853470   
4  youtube#comment     673093      21411     1265   2584166   

                                          textOriginal  parentCommentId  \
0  PLEASE LESBIAN FLAG I BEG YOU \n\nYou would rock it              NaN   
1     Apply mashed potato juice and mixed it with curd        3198066.0   
2                           69 missed calls from mars👽              NaN   
3                                                 Baaa              NaN   
4      you look like raven from phenomena raven no cap              NaN   

   likeCount                publishedAt                  updatedAt  \
0          0  2023-08-15 21:48:52+00:00  2023-08-15 21:48:52+00:00   
1          0  2023-10-02 13:08